In [1]:
import urllib
import json
import pandas as pd

In [ ]:
url = 'https://s3.amazonaws.com/stantemptesting/doc_voc_matrix.json'
d = json.loads(urllib.urlopen(url).read())

In [ ]:
data = pd.DataFrame(d.items())
data